In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dolly_cqa_df = pd.read_parquet('./data/dolly_cqa_prepared.parquet')
dolly_cqa_df

In [ ]:
def train_linear_regression_with_cv(df, embeddings_col, y_col):
    X = np.array(df[embeddings_col].tolist())
    X = X / np.linalg.norm(X, axis=1, keepdims=True)
    
    y = df[y_col].values
    
    kf = KFold(n_splits=7, shuffle=True, random_state=369)
    
    best_model = None
    best_error = float('inf')
    
    for train_index, val_index in kf.split(df):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        predictions = model.predict(X_val)
        error = mean_squared_error(y_val, predictions)
        
        if error < best_error:
            best_error = error
            best_model = model
    
    return best_model

In [ ]:
clf = train_linear_regression_with_cv(df=dolly_cqa_df, embeddings_col='embeddings_mini_lm', y_col='fk_score')

In [ ]:
def get_predictions(df, clf, embeddings_col, y_col):
    X = np.array(df[embeddings_col].tolist())
    X = X / np.linalg.norm(X, axis=1, keepdims=True)
    
    y = df[y_col].values

    y_pred = clf.predict(X)

    print('MSE:', mean_squared_error(y, y_pred))
    print('MAE:', mean_absolute_error(y, y_pred))
    print('R2:', r2_score(y, y_pred))

    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y, y=y_pred)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.grid(True)
    plt.show()

In [ ]:
get_predictions(df=dolly_cqa_df, clf=clf, embeddings_col='embeddings_mini_lm', y_col='fk_score')

In [ ]:
# dolly_cqa_df['score_easy'] = clf.predict_proba(X=np.array(dolly_cqa_df['embeddings_bge_large'].tolist()))[:,0]
# dolly_cqa_df['score_hard'] = clf.predict_proba(X=np.array(dolly_cqa_df['embeddings_bge_large'].tolist()))[:,1]
# dolly_cqa_df['score_medium'] = clf.predict_proba(X=np.array(dolly_cqa_df['embeddings_bge_large'].tolist()))[:,2]

# dolly_cqa_df['w_easy_embeddings'] = dolly_cqa_df['embeddings_bge_large'].apply(lambda x: x * clf.coef_[0])
# dolly_cqa_df['w_hard_embeddings'] = dolly_cqa_df['embeddings_bge_large'].apply(lambda x: x * clf.coef_[1])
# dolly_cqa_df['w_medium_embeddings'] = dolly_cqa_df['embeddings_bge_large'].apply(lambda x: x * clf.coef_[2])
# dolly_cqa_df['iw_easy_embeddings'] = dolly_cqa_df['embeddings_bge_large'].apply(lambda x: x / clf.coef_[0])
# dolly_cqa_df['iw_hard_embeddings'] = dolly_cqa_df['embeddings_bge_large'].apply(lambda x: x / clf.coef_[1])
# dolly_cqa_df['iw_medium_embeddings'] = dolly_cqa_df['embeddings_bge_large'].apply(lambda x: x / clf.coef_[2])

dolly_cqa_df['score'] = clf.predict(X=np.array(dolly_cqa_df['embeddings_mini_lm'].tolist()))
dolly_cqa_df['absolute_error'] = np.abs(np.array(dolly_cqa_df['fk_score']) - clf.predict(X=np.array(dolly_cqa_df['embeddings_mini_lm'].tolist())))
dolly_cqa_df['w_embeddings'] = dolly_cqa_df['embeddings_mini_lm'].apply(lambda x: x * clf.coef_)
dolly_cqa_df['iw_embeddings'] = dolly_cqa_df['embeddings_mini_lm'].apply(lambda x: x / clf.coef_)

In [ ]:
def get_cluster_stats(df, cluster_col, score_col):
    return df.groupby(cluster_col).agg(
        score_mean=pd.NamedAgg(column=score_col, aggfunc='mean'), 
        score_std =pd.NamedAgg(column=score_col, aggfunc='std')
    ).reset_index(drop=False).rename(columns={cluster_col: 'cluster_id'})

n_clusters = 20
seed=0
uw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['embeddings_mini_lm']])
w_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['w_embeddings']])
iw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['iw_embeddings']])

dolly_cqa_df['uw_kmeans'] = uw_kmeans.labels_
dolly_cqa_df['w_kmeans'] = w_kmeans.labels_
dolly_cqa_df['iw_kmeans'] = iw_kmeans.labels_

# cs_unweighted = get_cluster_stats(dd2, 'unweighted_kmeans', 'score')
# cs_weighted = get_cluster_stats(dd2, 'weighted_kmeans', 'score')
# cs_inverse_weighted = get_cluster_stats(dd2, 'inverse_weighted_kmeans', 'score')

In [ ]:
cluster_df = dolly_cqa_df[['uw_kmeans', 'w_kmeans', 'iw_kmeans', 'score', 'absolute_error']]
cluster_df = pd.melt(cluster_df, id_vars=['score', 'absolute_error'], var_name='weighting', value_name='cluster_id')
cluster_df = cluster_df.groupby(['weighting', 'cluster_id']).agg({'score': ['mean', 'var'], 'absolute_error': ['mean', 'var']}).reset_index()
cluster_df.columns = ['weighting', 'cluster_id', 'mean_score', 'var_score', 'mean_absolute_error', 'var_absolute_error']

In [ ]:
cluster_df

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='mean_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='mean_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Mean Score by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='var_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='var_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Variance of Scores by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='mean_absolute_error', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='mean_absolute_error', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Mean Absolute Error by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='var_absolute_error', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='var_absolute_error', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Variance of Absolute Errors by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
def plot_cluster_scores(df):
  
    cluster_columns = ['uw_kmeans', 'w_kmeans', 'iw_kmeans']
    
    # Set up the matplotlib figure
    num_clusters = len(cluster_columns)
    fig, axs = plt.subplots(num_clusters, 1, figsize=(8, 4*num_clusters))
    
    for i, cluster_column in enumerate(cluster_columns):
        ax = axs[i]
        sns.countplot(data=df, x=cluster_column, hue='fk_score_categ', ax=ax)
        ax.set_title(f'Score Counts by {cluster_column}')
        ax.set_xlabel('Cluster')
        ax.set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cluster_scores(dolly_cqa_df)

In [ ]:
dolly_cqa_df['fk_score'].hist(bins=20)

In [ ]:
len(dolly_cqa_df[dolly_cqa_df['fk_score'] <= 7])

In [ ]:
aspect_embedding = np.mean(dolly_cqa_df[dolly_cqa_df['fk_score'] <= 7]['embeddings_mini_lm'])

aspect_embedding = aspect_embedding / np.linalg.norm(aspect_embedding)

dolly_cqa_df['w_embeddings'] = dolly_cqa_df['embeddings_mini_lm']
dolly_cqa_df['iw_embeddings'] = dolly_cqa_df['embeddings_mini_lm']

alpha = 1.75
for i, embedding in enumerate(dolly_cqa_df['embeddings_mini_lm']):
    embedding = embedding / np.linalg.norm(embedding)
    projection = np.dot(embedding, aspect_embedding.T) * aspect_embedding
    projection = projection.reshape(-1,)
    dolly_cqa_df['w_embeddings'].iloc[i] = embedding + alpha * projection
    dolly_cqa_df['iw_embeddings'].iloc[i] = embedding - alpha * projection


In [ ]:
n_clusters = 20
seed=0
uw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['embeddings_mini_lm']])
w_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['w_embeddings']])
iw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['iw_embeddings']])

dolly_cqa_df['uw_kmeans'] = uw_kmeans.labels_
dolly_cqa_df['w_kmeans'] = w_kmeans.labels_
dolly_cqa_df['iw_kmeans'] = iw_kmeans.labels_

cluster_df = dolly_cqa_df[['uw_kmeans', 'w_kmeans', 'iw_kmeans', 'score', 'absolute_error']]
cluster_df = pd.melt(cluster_df, id_vars=['score', 'absolute_error'], var_name='weighting', value_name='cluster_id')
cluster_df = cluster_df.groupby(['weighting', 'cluster_id']).agg({'score': ['mean', 'var'], 'absolute_error': ['mean', 'var']}).reset_index()
cluster_df.columns = ['weighting', 'cluster_id', 'mean_score', 'var_score', 'mean_absolute_error', 'var_absolute_error']

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='mean_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='mean_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Mean Score by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='var_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='var_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Variance of Scores by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
plot_cluster_scores(dolly_cqa_df)